In [5]:
import pandas as pd 

In [6]:
train_df = pd.read_csv("/Users/anishkamukherjee/Documents/tweet_sentiment/data/train.csv", encoding='unicode_escape')
train_df.head(5)

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [7]:
test_df = pd.read_csv("/Users/anishkamukherjee/Documents/tweet_sentiment/data/test.csv", encoding='unicode_escape')
test_df.head(5)

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [8]:
train_df = train_df.dropna(axis=0)
test_df = test_df.dropna(axis=0)
train_df.isna().sum(), test_df.isna().sum()

(textID              0
 text                0
 selected_text       0
 sentiment           0
 Time of Tweet       0
 Age of User         0
 Country             0
 Population -2020    0
 Land Area (Km²)     0
 Density (P/Km²)     0
 dtype: int64,
 textID              0
 text                0
 sentiment           0
 Time of Tweet       0
 Age of User         0
 Country             0
 Population -2020    0
 Land Area (Km²)     0
 Density (P/Km²)     0
 dtype: int64)

In [9]:
len(train_df.text), len(train_df.sentiment), len(test_df.text), len(test_df.sentiment)

(27480, 27480, 3534, 3534)

In [10]:
# remove column space 
def remove_col_space(df: pd.DataFrame):
    if type(df) == pd.DataFrame:
        df.columns = [col.lower().replace(' ', '_') for col in df.columns]
        return df.columns

In [11]:
train_df_cols = train_df.columns = remove_col_space(train_df)
test_df_cols = test_df.columns = remove_col_space(test_df)
train_df_cols, test_df_cols

(Index(['textid', 'text', 'selected_text', 'sentiment', 'time_of_tweet',
        'age_of_user', 'country', 'population_-2020', 'land_area_(km²)',
        'density_(p/km²)'],
       dtype='object'),
 Index(['textid', 'text', 'sentiment', 'time_of_tweet', 'age_of_user',
        'country', 'population_-2020', 'land_area_(km²)', 'density_(p/km²)'],
       dtype='object'))

In [12]:
def remove_cols(df: pd.DataFrame, *args) -> pd.DataFrame:
    df = df.drop(list(args), axis=1)
    return df

In [13]:
new_train_df = remove_cols(train_df, 'textid', 'selected_text', 'time_of_tweet', 'age_of_user', 'country', 'population_-2020', 'land_area_(km²)', 'density_(p/km²)')
new_train_df.head(5)
new_train_df.isna().sum()

text         0
sentiment    0
dtype: int64

In [14]:
new_test_df = remove_cols(test_df, 'textid', 'time_of_tweet', 'age_of_user', 'country', 'population_-2020', 'land_area_(km²)', 'density_(p/km²)')
new_test_df.head(5)
new_test_df.isna().sum()

text         0
sentiment    0
dtype: int64

In [15]:
# make the case in lower letters
def lower_text(df: pd.DataFrame, col_name: str) -> pd.Series:
    df[col_name] = df[col_name].str.lower()
    return df[col_name]

In [16]:
lower_text(new_train_df, "text")
new_train_df.head(5)

,text,sentiment
0,"i`d have responded, if i were going",neutral
1,sooo sad i will miss you here in san diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"sons of ****, why couldn`t they put them on t...",negative


In [17]:
lower_text(new_test_df, "text")
new_test_df.head(5)

,text,sentiment
0,last session of the day http://twitpic.com/67ezh,neutral
1,shanghai is also really exciting (precisely -...,positive
2,"recession hit veronique branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - i like it!!,positive


In [18]:
# remove any html tags if present
import re

def remove_html(col):
    if isinstance(col, str):
        pattern = re.compile(r'<,*?>')
        return pattern.sub('', col)
    else:
        return col

In [19]:
new_train_df["text"] = new_train_df["text"].apply(remove_html)
new_test_df["text"] = new_test_df["text"].apply(remove_html)

In [20]:
def remove_urls(text):
    if isinstance(text, str):
        pattern = re.compile(r'http://\S+|https://\S+|www\.\S+')
        return pattern.sub('', text)
    else:
        return text

In [21]:
new_train_df["text"] = new_train_df["text"].apply(remove_urls)
new_test_df["text"] = new_test_df["text"].apply(remove_urls)

In [22]:
new_test_df.text.head(5)

0                            last session of the day  
1     shanghai is also really exciting (precisely -...
2    recession hit veronique branquinho, she has to...
3                                          happy bday!
4                                        - i like it!!
Name: text, dtype: object

In [23]:
# remove punctuations 
import string
exclude = string.punctuation

def remove_punctuations(text):
    if isinstance(text, str):
        return text.translate(str.maketrans('', '', exclude))
    else:
        return text

In [24]:
new_train_df["text"] = new_train_df["text"].apply(remove_punctuations)
new_test_df["text"] = new_test_df["text"].apply(remove_punctuations)

In [25]:
new_test_df.text.head(5)

0                            last session of the day  
1     shanghai is also really exciting precisely  s...
2    recession hit veronique branquinho she has to ...
3                                           happy bday
4                                            i like it
Name: text, dtype: object

In [26]:
# some columns may contain spelling errors. so we have to correct them.
from textblob import TextBlob

def correct_spellings(text):
    if isinstance(text, str):
        tblob = TextBlob(text)
        return str(tblob.correct())  
    return text

In [27]:
# new_train_df["text"] = new_train_df["text"].apply(correct_spellings)
# new_test_df["text"] = new_test_df["text"].apply(correct_spellings)

'''This code took a lot of time to execute so I'm not gonna waste my time to check if there are any typos in any of the tweets or not.'''

"This code took a lot of time to execute so I'm not gonna waste my time to check if there are any typos in any of the tweets or not."

In [28]:
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anishkamukherjee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
new_train_df["text"] = [word_tokenize(str(col)) if isinstance(col, str) else [] for col in new_train_df["text"]]

In [ ]:
new_train_df["text"]

0                [id, have, responded, if, i, were, going]
1        [sooo, sad, i, will, miss, you, here, in, san,...
2                             [my, boss, is, bullying, me]
3                      [what, interview, leave, me, alone]
4        [sons, of, why, couldnt, they, put, them, on, ...
                               ...                        
27476    [wish, we, could, come, see, u, on, denver, hu...
27477    [ive, wondered, about, rake, to, the, client, ...
27478    [yay, good, for, both, of, you, enjoy, the, br...
27479                            [but, it, was, worth, it]
27480    [all, this, flirting, going, on, the, atg, smi...
Name: text, Length: 27480, dtype: object

In [ ]:
new_test_df["text"] = [word_tokenize(str(col)) if isinstance(col, str) else [] for col in new_test_df["text"]]

In [ ]:
new_test_df["text"]

0       [Last, session, of, the, day, http, :, //twitp...
1       [Shanghai, is, also, really, exciting, (, prec...
2       [Recession, hit, Veronique, Branquinho, ,, she...
3                                        [happy, bday, !]
4       [http, :, //twitpic.com/4w75p, -, I, like, it,...
                              ...                        
3529    [its, at, 3, am, ,, im, very, tired, but, i, c...
3530    [All, alone, in, this, old, house, again, ., T...
3531    [I, know, what, you, mean, ., My, little, dog,...
3532    [_sutra, what, is, your, next, youtube, video,...
3533    [http, :, //twitpic.com/4woj2, -, omgssh, ang,...
Name: text, Length: 3534, dtype: object

In [ ]:
len(new_train_df["text"]), len(new_train_df["sentiment"]), len(new_test_df["text"]), len(new_test_df["sentiment"])

(27481, 27481, 4815, 4815)

### Tokenization from scratch

In [ ]:
# word to int
%time

def word2vec_gen(df: pd.DataFrame, col_name: str):
    current_idx = 1
    word2vec = {}
    for words in df[col_name]:
        for word in words:
            if word not in word2vec:
                word2vec[word] = current_idx
                current_idx += 1
    return word2vec

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 8.11 μs


In [ ]:
word2vec_train = word2vec_gen(new_train_df, "text")

In [ ]:
train_seq = [[word2vec_train[word] for word in words] for words in new_train_df["text"]]
test_seq = [[word2vec_train.get(word, 0) for word in words] for words in new_test_df["text"]]

train_seq, test_seq

([[1, 2, 3, 4, 5, 6, 7],
  [8, 9, 5, 10, 11, 12, 13, 14, 15, 16],
  [17, 18, 19, 20, 21],
  [22, 23, 24, 21, 25],
  [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
  [39, 40, 41, 42, 34, 43, 44, 45, 33, 46],
  [47, 48, 42, 34, 49, 50, 51, 52, 53, 19, 54, 55, 56, 57],
  [58, 59],
  [60, 27, 12],
  [61, 62, 63, 64, 65, 66, 67, 19, 68, 69],
  [70,
   71,
   70,
   5,
   72,
   73,
   74,
   75,
   5,
   76,
   34,
   77,
   50,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   17,
   85,
   56,
   86],
  [5, 87, 87, 88, 34, 89, 72, 90, 91, 92, 93],
  [17, 94, 19, 95, 96, 97, 33, 98],
  [5, 99, 73, 100, 73, 101, 102, 103, 5, 104, 17, 105],
  [106, 106, 107, 34, 108, 109],
  [110, 111, 5, 112, 113],
  [114, 115, 73, 116, 117, 70, 36, 118, 119],
  [120,
   121,
   122,
   42,
   34,
   123,
   124,
   125,
   56,
   126,
   17,
   127,
   128,
   129,
   4,
   5,
   130,
   2,
   131,
   132,
   33,
   133,
   134,
   135],
  [19, 136, 137, 138, 82, 83, 11, 139, 140],
  [141, 64, 142

### Tokenization with Tensorflow

In [ ]:
# Another easy way of word 2 vector gen using tensorflow
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 20000)
tokenizer.fit_on_texts(new_train_df["text"])
train_sequences_tf = tokenizer.texts_to_sequences(new_train_df["text"])
test_sequences_tf = tokenizer.texts_to_sequences(new_test_df["text"])

In [ ]:
train_sequences_tf, test_sequences_tf

([[290, 16, 10298, 68, 1, 120, 46],
  [413, 115, 1, 58, 89, 7, 88, 10, 1459, 2227],
  [5, 1297, 9, 10299, 15],
  [56, 1112, 341, 15, 494],
  [2758, 12, 110, 385, 86, 317, 127, 14, 3, 6992, 63, 202, 555],
  [67, 10300, 5389, 11, 3, 156, 5390, 3043, 14, 1298],
  [3385, 10301, 11, 3, 295, 35, 102, 80, 112, 9, 28, 2228, 6, 10302],
  [571, 556],
  [404, 12, 7],
  [2229, 248, 55, 20, 2384, 3386, 539, 9, 18, 1299],
  [77,
   84,
   77,
   1,
   43,
   2,
   23,
   3044,
   1,
   3387,
   3,
   3858,
   35,
   6993,
   600,
   13,
   165,
   123,
   76,
   34,
   5,
   701,
   6,
   308],
  [1, 57, 57, 36, 3, 286, 43, 798, 122, 1085, 2385],
  [5, 10303, 9, 511, 10304, 939, 14, 5391],
  [1, 74, 2, 38, 2, 323, 118, 19, 1, 301, 5, 1086],
  [592, 592, 49, 3, 4471, 10305],
  [1681, 83, 1, 65, 2759],
  [10306, 215, 2, 4472, 6994, 77, 63, 1535, 612],
  [137,
   100,
   195,
   11,
   3,
   637,
   311,
   150,
   6,
   3388,
   5,
   324,
   234,
   6995,
   68,
   1,
   117,
   16,
   4,
   1595,
  

In [ ]:
df_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences_tf)
df_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences_tf)
df_train.shape[1]

33

In [ ]:
len(tokenizer.word_index)

27912

In [ ]:
new_train_df.sentiment.value_counts()

sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64

In [ ]:
new_train_df.isna().sum(), new_test_df.isna().sum()

(text         0
 sentiment    0
 dtype: int64,
 text            0
 sentiment    1281
 dtype: int64)

In [ ]:
# OneHotEncoding the target values

from sklearn.preprocessing import OneHotEncoder



encoder = OneHotEncoder()
encoded_train_sentiment = encoder.fit_transform(new_train_df[["sentiment"]])

encoded_test_sentiment = encoder.transform(new_test_df[["sentiment"]])

new_train_df["sentiment"] = list(encoded_train_sentiment)
new_test_df["sentiment"] = list(encoded_test_sentiment)

ValueError: Found unknown categories [nan] in column 0 during transform